In [1]:
import gpt_2
from utils import *
import tensorflow as tf
from sklearn.cross_validation import train_test_split
import time
import random
import os

/usr/local/lib/python3.6/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
trainset = sklearn.datasets.load_files(container_path = 'data', encoding = 'UTF-8')
trainset.data, trainset.target = separate_dataset(trainset,1.0)
print (trainset.target_names)
print (len(trainset.data))
print (len(trainset.target))

['negative', 'positive']
10662
10662


In [3]:
concat = ' '.join(trainset.data).split()
vocabulary_size = len(list(set(concat)))
data, count, dictionary, rev_dictionary = build_dataset(concat, vocabulary_size)
print('vocab from size: %d'%(vocabulary_size))
print('Most common words', count[4:10])
print('Sample data', data[:10], [rev_dictionary[i] for i in data[:10]])

vocab from size: 20332
Most common words [('film', 1453), ('movie', 1270), ('one', 727), ('like', 721), ('story', 477), ('much', 386)]
Sample data [2417, 587, 7006, 10158, 475, 143, 5, 187, 4373, 4374] ['patient', 'lovely', 'hush', 'reward', 'thoughtful', 'emotional', 'movie', 'experience', 'ray', 'liotta']


In [4]:
GO = dictionary['GO']
PAD = dictionary['PAD']
EOS = dictionary['EOS']
UNK = dictionary['UNK']

In [5]:
from tensorflow.contrib.training import HParams

params = HParams(
    n_vocab = len(dictionary),
    n_ctx = 512,
    n_embd = 256,
    n_head = 8,
    n_layer = 8,
)
maxlen = 100
batch_size = 16
learning_rate = 1e-3

In [6]:
class Model:
    def __init__(self):
        self.X = tf.placeholder(tf.int32, [None, maxlen])
        self.Y = tf.placeholder(tf.int32, [None])
        logits = gpt_2.model(params, self.X)
        logits = tf.reduce_mean(logits['logits'], axis=1)
        self.logits = tf.layers.dense(logits, 2)
        self.cost = tf.reduce_mean(
            tf.nn.sparse_softmax_cross_entropy_with_logits(
                logits = self.logits, labels = self.Y
            )
        )
        self.optimizer = tf.train.AdamOptimizer(learning_rate).minimize(
            self.cost
        )
        correct_pred = tf.equal(
            tf.argmax(self.logits, 1, output_type = tf.int32), self.Y
        )
        self.accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [7]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Model()
sess.run(tf.global_variables_initializer())

In [8]:
vectors = str_idx(trainset.data,dictionary,maxlen)
train_X, test_X, train_Y, test_Y = train_test_split(vectors, trainset.target,test_size = 0.2)

In [9]:
from tqdm import tqdm
import time

EARLY_STOPPING, CURRENT_CHECKPOINT, CURRENT_ACC, EPOCH = 3, 0, 0, 0

while True:
    lasttime = time.time()
    if CURRENT_CHECKPOINT == EARLY_STOPPING:
        print('break epoch:%d\n' % (EPOCH))
        break

    train_acc, train_loss, test_acc, test_loss = 0, 0, 0, 0
    pbar = tqdm(
        range(0, len(train_X), batch_size), desc = 'train minibatch loop'
    )
    for i in pbar:
        batch_x = train_X[i : min(i + batch_size, train_X.shape[0])]
        batch_y = train_Y[i : min(i + batch_size, train_X.shape[0])]
        acc, cost, _ = sess.run(
            [model.accuracy, model.cost, model.optimizer],
            feed_dict = {
                model.Y: batch_y,
                model.X: batch_x
            },
        )
        assert not np.isnan(cost)
        train_loss += cost
        train_acc += acc
        pbar.set_postfix(cost = cost, accuracy = acc)
        
    pbar = tqdm(range(0, len(test_X), batch_size), desc = 'test minibatch loop')
    for i in pbar:
        batch_x = test_X[i : min(i + batch_size, test_X.shape[0])]
        batch_y = test_Y[i : min(i + batch_size, test_X.shape[0])]
        acc, cost = sess.run(
            [model.accuracy, model.cost],
            feed_dict = {
                model.Y: batch_y,
                model.X: batch_x
            },
        )
        test_loss += cost
        test_acc += acc
        pbar.set_postfix(cost = cost, accuracy = acc)

    train_loss /= len(train_X) / batch_size
    train_acc /= len(train_X) / batch_size
    test_loss /= len(test_X) / batch_size
    test_acc /= len(test_X) / batch_size

    if test_acc > CURRENT_ACC:
        print(
            'epoch: %d, pass acc: %f, current acc: %f'
            % (EPOCH, CURRENT_ACC, test_acc)
        )
        CURRENT_ACC = test_acc
        CURRENT_CHECKPOINT = 0
    else:
        CURRENT_CHECKPOINT += 1

    print('time taken:', time.time() - lasttime)
    print(
        'epoch: %d, training loss: %f, training acc: %f, valid loss: %f, valid acc: %f\n'
        % (EPOCH, train_loss, train_acc, test_loss, test_acc)
    )
    EPOCH += 1

train minibatch loop:   0%|          | 1/534 [00:00<01:40,  5.30it/s, accuracy=0.438, cost=0.699]

epoch: 0, pass acc: 0.000000, current acc: 0.491233
time taken: 108.65052771568298
epoch: 0, training loss: 1.405971, training acc: 0.496541, valid loss: 0.698534, valid acc: 0.491233



train minibatch loop:   0%|          | 1/534 [00:00<01:40,  5.31it/s, accuracy=0.438, cost=0.699]

time taken: 108.08198261260986
epoch: 1, training loss: 0.696155, training acc: 0.508735, valid loss: 0.698582, valid acc: 0.491233



train minibatch loop:   0%|          | 1/534 [00:00<01:40,  5.32it/s, accuracy=0.438, cost=0.731]

time taken: 108.02254128456116
epoch: 2, training loss: 0.706807, training acc: 0.502169, valid loss: 0.716543, valid acc: 0.491233



test minibatch loop: 100%|██████████| 134/134 [00:07<00:00, 18.45it/s, accuracy=0.8, cost=0.625]  

time taken: 107.98197817802429
epoch: 3, training loss: 0.707655, training acc: 0.498065, valid loss: 0.707726, valid acc: 0.491233

break epoch:4



In [10]:
real_Y, predict_Y = [], []

pbar = tqdm(
    range(0, len(test_X), batch_size), desc = 'validation minibatch loop'
)
for i in pbar:
    batch_x = test_X[i : min(i + batch_size, test_X.shape[0])]
    batch_y = test_Y[i : min(i + batch_size, test_X.shape[0])]
    predict_Y += np.argmax(
        sess.run(
            model.logits, feed_dict = {model.X: batch_x, model.Y: batch_y}
        ),
        1,
    ).tolist()
    real_Y += batch_y

validation minibatch loop: 100%|██████████| 134/134 [00:07<00:00, 18.52it/s]


In [11]:
print(metrics.classification_report(real_Y, predict_Y, target_names = trainset.target_names))

             precision    recall  f1-score   support

   negative       0.00      0.00      0.00      1094
   positive       0.49      1.00      0.66      1039

avg / total       0.24      0.49      0.32      2133



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
